# Flask Basics, lesson three!!

## Forms

![SegmentLocal](https://i.imgur.com/zm5ROlS.gif?noredirect "segment")

We will now look into ways to create forms using flask 

The first way, which is the very simple way, is making HTML forms the way we know, and then connecting our Flask Application

The easiest way to show this is in a large example!!

In this example we will create a website that has a home page, a nav bar that takes you to the home page when clicked, a survey page that takes the name of someone who wants to join, a thank you for signing up page, as well as a custom 404 error!!

In [1]:
## Flask App
from flask import Flask, render_template, request
## request is another function we are getting that will help us with making our thank you page
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/signupform')
def signup_form():
    return render_template('signup.html')

@app.route('/thank_you')
def thank_you():
    first = request.args.get('first')
    last = request.args.get('last')
    ## These go to where the get was used in a form and find which one had the label first or last, and grab the answer provided
    return render_template('thankyou.html', first = first, last = last)

@app.errorhandler(404)
def page_not_found(e):
    ##The e is custom since it stands for error
    return render_template('404.html'), 404

if __name__ == '__main__':
    app.run(debug)

In [ ]:
## base.html
<!DOCTYPE html>
<html lang = "en" dir = "ltr">
    <head>
        <meta charset = “utf=8”>
        <title>Puppy Rock</title>
        Bootstrap links!
    </head>
    <body>
        <nav class = “navbar navbar-expand-lg navbar-light bg-light”>
        <a class = “navbar-brand” href = “{{ url_for(‘index’)}}”>Home Page</a>
        </nav>
        
        {% block content %}
        
        {% endblock %}
    <body>
</html>

In [ ]:
## index.html

{% extends "base.html" %}

{% block content %}
    <div class = "jumbotron">
        <p>Welcome to Puppy Rock!</p>
        <p>Wanna sign up for our pupp band??</p>
            <a href = "{{url_for('signup_form')}}">Sign up for auditions here</a>
    </div>
{% endblock %}

In [ ]:
## Signup_form.html
{% extends "base.html" %}

{% block content %}
    <div class = "jumbotron">
        <h1>Welcome to the sign up page!</h1>
        <p>Fill out the form</p>
            <form action = "{{url_for('thank_you')}}">
                <label for = "first">First name:</label>
                <input type = "text" name = "first">
                <label for = "last">Last name:</label>
                <input type = "text" name = "last">
                <input type = "submit" value = "Submit Form">
            </form>
    </div>
{% endblock %}

In [ ]:
## thank_you.html 
{% extends "base.html" %}

{% block content %}
    <div class = "jumbotron">
        <h1>Thanks for signing up! {{first}} {{last}} </h1>
    </div>
{% endblock %}

In [ ]:
## 404.html
{% extends "base.html" %}

{% block content %}
    <div class = "jumbotron">
        <p>Sorry could not find that page!</p>
    </div>
{% endblock %}

Source Jose Portilla

## More Advanced Forms 

![SegmentLocal](https://media.tenor.com/images/11735777cd03fe010787526ce1704027/tenor.gif "segment")

We can use the flask_wtf and WTForms packages to create forms from our flask python scripts 

But we need certain things:

We need to configure a secret key for security

We need to create a WTForm class, and fields for each part of the form

We need to set up a view function, and add methods = ['GET', 'POST']

Create and instance of Form Class, and handle form submission

Here is an example, with comments to break it down!

In [ ]:
## Basic.py
from flask import Flask, render_template
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mysecretkey'
## Setting configuration of secret key to an arbitrary string 

Class InfoForm(FlaskForm):
    ##InfoForm is the class of choice, inheriting from FlaskForm, which we imported from flask_wtf
    breed = StringField("What breed are you?")
    submit = SubmitField('Submit')
    ## InfoForm has two attributes, that we just created above

@app.route('/', methods = ['GET', 'POST'])
## So we are not able to get and post these forms
def index():
    breed = False
    ## different from the breed above
    form = InfoForm()
    if form.validate_on_submit():
        ##if the submission was valid
        breed = form.breed.data
        ##gets what was entered in the form for breed
        form.breed.data = ''
        ##resets the form.breed.data that we just copied back to an empty string
    return render_template('home.html', form = form, breed = breed)

if __name__ == '__main__':
    app.run(debug = True)

In [ ]:
## home.html
<p>
{% if breed %}
    The breed you entered is {{breed}}
    update it in the form below:
{% else %}
    Please enter your breed: 
{% endif %}
</p>

<form method = 'POST'>
    {{form.hidden_tag()}}
    {{form.breed.label}}{{form.breed()}}
    {{form.submit()}}
    ## We sent into the template a form with two attributes, and the attributes and methods inherited from FlaskForm
    ## breed.label is the question the form poses, and form.breed() is where you type the answer to the question
</form>

Source Jose Portilla

### Fields

![SegmentLocal](https://media3.giphy.com/media/xUA7aLSKFAm4nmaeU8/giphy.gif "segment")

Now that we have a better idea of forms, we will begin to work with form fields more

If there is an HTML form field, then there is a wtforms class that you can import to match it

wtforms also has a tool called validators you can import!

These validators check the form data for specific things, such as making a field required to be filled out

Here is an example to break it down:

In [ ]:
## Basic.py
from flask import Flask, render_template, session, redirect, url_for
## session and redirect are new and you will see how to use them in this example
from flask_wtf import FlaskForm
from wtforms import (StringField, BooleanField, DateTimeField, RadioField, 
                     SelectField, TextField, TextAreaField, SubmitField)
## The parentheses allow all the imports to be on seperate lines without error
from wtforms.validators import DataRequired
## this is where validators are introduced, DataRequired is an example of one, but there are many others as well

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mykey'

Class InfoForm(FlaskForm):
    breed = StringField('What breed are you?', validators = [DataRequired()])
    ## This is the first part of the form, a StringField, so users are presented with the question "What breed are you?" and then provide a string answer
    ## DataRequired is used here as a validator, therefore it will make sure data is recieved for this question
    neutered = BooleanField("Have you been neutered?")
    ##This is another form field, BooleanField, so the user is presented with the question, then can check the box for yes, or leave it unchecked for no
    mood = RadioField('Please choose your mood:', choices = [('mood_one', 'Happy'), ('mood_two', 'Excited')])
    ## Another form field type, RadioField. For the choices, the first part, such as mood_one or mood_two, of each is for back_end
    ## The second part, such as Happy or Excited, is what is presented to the user for each radio button, as the label
    food_choice = SelectField('Pick you favorite food:' choices = [('chi', 'Chicken'), ('bf', 'Beef'), ('fish', 'Fish')])
    ## this works the same as the RadioField, but is presented in a different style than the radio buttons
    feedback = TextAreaField()
    ## TextAreaField is provided where the user can write whatever they want
    submit = SubmitField('Submit')

@app.route('/', methods = ['GET', 'POST'])
def index():
    form = InfoForm()
    if form.validate_on_submit():
        ##if form is valid on submission
        session['breed'] = form.breed.data
        ##session is a way to store data for a specific amount of time, from when a user logs in to when they log out
        ## Treat it like a dictionary, using 'breed' as a key
        session['neutered'] = form.neutered.data
        session['mood'] = form.mood.data
        session['food'] = form.food_choice.data
        session['feedback'] = form.feedback.data
        return redirect(url_for('thankyou'))
        ##This redirection to the thankyou page only happens upon valid submission of the form
    return render_template('index.html', form = form)

@app.route('/thankyou')
def thankyou():
    return render_template('thankyou.html')

if __name__ == '__main__':
    app.run(debug = True)

In [ ]:
## Index.html
<h1> Welcome to the Puppy Survey</h1>
<form method = 'post'>
    {{form.hidden_tag()}}
    {{form.breed.label}}{{form.breed}}
    <br>
    {{form.neutered.label}}{{form.neutered}}
    <br>
    {{form.food_choice.label}}{{form.food_choice}}
    <br>
    {{form.mood.label}}{{form.mood}}
    <br>
    Any other feedback?
    <br>
    {{form.feedback}}
    <br>
    {{form.submit}}
    ## The <br> between them breaks up the text by line]

In [ ]:
## thankyou.html
<h1>Thank you. Here ios the info you have us:</h1>
<ul>
    <li>Breed: {{sessopn['breed']}} </li>
    <li>Neutered: {{session['neutered']}} </li>
    <li>Mood: {{session['mood']}} </li>
    <li>Food: {{session['food']}} </li>
    <li>Feedback: {{session['feedback']}} </li>
</ul>
## In the case of mood and food, what is printed is the backend balue, mood_one or mood_two, rather than Happy or Excited

Source Jose Portilla

### Flashing Messages!

![SegmentLocal](https://media.giphy.com/media/OQJQuRHDVJ1a8/giphy.gif "segment")

On web applications, there are many instances when you want to send a message to a user that you don't permanently want there

A pop up message, where you flash the message to the users and the users then have the option to close the message 

This will create something that looks like one of these:

![SegmentLocal](https://javabeat.net/wp-content/uploads/2014/06/Bootstrap-Dismissal-Alert-Messages-Example.jpg "segment")

In [ ]:
##basic.py
from flask import Flask, render_template, flash, session, redirect, url_for
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField

app = Flask(__name__)

app.config['SECRET_KEY'] = 'mykey'

class SimpleForm(FlaskForm):
    submit = SubmitField('Click Me.')
    
@app.route('/', methods = ['GET', 'POST'])
def index():
    form = SimpleForm()
    if form.validate_on_submit():
        ##if they clicked the button
        flash('you just clicked the button!')
        return redirect(url_for('index'))
    return render_template('index.html', form=form)

if __name__ == '__main__':
    app.run(debug = True)

In [ ]:
##index.html
## We will use a bit of bootstrap code!!! It can be found in the alerts portion of the documentation, under dismissing 
<!DOCTYPE html>
<html lang = "en" dir = "ltr">
    <head>
        <meta charset = "utf-8">
        ## Bootstrap links for jquery and JS!!!!
        <title></title>
    </head>
    <body>
        {% for mess in get_flashed_messages() %}
        ## get_flashed_messages is what gets passed back when you call flash in the python code, 
        ##you loop through in case there is more than one message you want to flash
       ## begininnging of bootstrap code!!
        <div class = "alert alert-warning alert-dismissible fade show" role = "alert">
           <button type = "button" class = "fade close" data-dismiss = "alert" aria-label = "close">
                <span aria-hidden = "true">&times;</span>
           </button>
            {{mess}}
        </div>
        ## end of boostrap code!
        ## you don't need to memorize this or anything, just have an understanding of what it is doing!
        {% endfor %}
        <form method = "POST">
            {{form.hidden_tag()}}
            {{form.submit()}}
        </form>
    </body>
</html>

Source Jose Portilla

For this code, a button will appear on the home page, that says "Click Me"

When you click it, a pop up appears from the top of the screen that says "you just clicked the button!" 

This pop up has an X on it, so the user can click that to get the pop up to fade away !!

![SegmentLocal](https://media1.tenor.com/images/53285ed070aea4f963141d04a1d23fae/tenor.gif?itemid=7342741 "segment")

# Homework

Add forms to your flask application!!